In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [19]:
import re
import numpy as np
import base64
import requests
from tqdm import tqdm
from PIL import Image
import fitz
import io
import pandas as pd
import random
import ast
from openai import OpenAI
import json
import fitz
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import base64
gpt = OpenAI()

# 1. 주택 청약 FAQ 문서 - 파싱 

In [ ]:
# from utils.util import get_project_root
# path = get_project_root() / "src" / "data" / "housing_pre_promise"/ "housing_faq.json"
# with open(path, "r", encoding="utf-8") as f:
#     parsing_list = json.load(f)
# len(parsing_list)


233

In [ ]:
# system = '''당신은 주어진 2개의 문서로부터 사용자의 가능한 질문과 답변의 쌍 5개를 생성해야 합니다.

# 1. 2개의 문서는 서로 이어지는 문서이므로 문맥이 순차적으로 이어지고 있음을 감안하세요.
# 2. content는 본문이고 source는 출처입니다.
# 3. 주어진 2개의 문서로부터 가능한 질문과 답변의 쌍을 5개 생성하세요.
# 4. Q1과 A1, Q2와 A2, Q3과 A3, Q4과 A5, Q4과 A5 이렇게 작성하면됩니다.
# 5. 각 답변의 뒤에는 출처 문서들을 리스트 형태로 작성하세요. 다수의 문서를 동시에 참고할 수 있습니다.
# 6. 주어진 문서에 없는 내용은 작성하지 마십시오. 오직 'content' 안에 있는 내용만 답변할 수 있습니다.
# 7. 답변은 오직 'content' 안에 있는 내용으로만 답변하지만 최대한 풍부하고 길게 작성하세요. 이는 매우 중요합니다.
# 8. 다수의 문서를 동시에 참고하여 답변하는 것이 가능합니다.
# 9. 답변할 때 문서에 해당 항목이 약관의 몇 조인지 명확한 상황이라면 답변할 때 무조건 이를 언급하면 좋겠습니다. 제10조(보험금의 지급사유)에 따르면~ 이런 것처럼 몇 조인지 언급해서 답변해주시기 바랍니다. 이는 반드시 지켜져야 합니다.
# 10. 너무 억지스럽게 지엽적인 질문은 금지하겠습니다. 이것은 기계독해 문제가 아닙니다. 예를 들어 아래와 같은 질문은 지양하십시오.
# ex) 보험금 지급에 관한 세부규정 중, 피보험자가 장해분류표 중 동일한 재해 또는 원인으로 여러 신체부위의 장해지급률을 더하여 50% 이상인 장해상태가 되었을 경우 보험료 납입 면제 조건은 무엇인가요?
# 11. 위와 같이 복잡한 질문이 아니라 일반 사용자가 할만한 질문을 던지십시오.
# 12. 주어진 문서에서 예시, 표, 글머리 기호로 표시된 것들을 FAQ로 만드는 것을 주저하지 마십시오. 이들은 FAQ로 만들때 가장 먼저 우선순위를 가집니다. 이들이 가장 중요합니다.

# 시작!
# '''

# my_list = parsing_list

# user_prompt = []

# # 리스트를 순회하면서 각 원소와 그 다음 원소만 출력
# for i in range(len(my_list) - 1):  # 끝에서 두 번째 원소까지만 순회하도록 변경
#     user_prompt.append('입력: ' + str(my_list[i]) + ' ' + str(my_list[i+1]) + '\n답변:')

# qa_result_lst = []

# previous_question = ''
# file_path = get_project_root() / "src" / "data" / "housing_pre_promise" / "chunk_list"
# with open(file_path, 'w', encoding='utf-8') as file:
#     for user in tqdm(user_prompt):

#         if len(previous_question) > 1:
#            user = user + '\n단 아래의 질문과 답변과 유사한 내용은 피하십시오. 이미 갖고 있는 데이터입니다.\n' + previous_question + '\n위 내용과 거의 중복되고 유사한 질문 답변은 필요없습니다.'

#         completion = gpt.chat.completions.create(
#         model="gpt-4-1106-preview",
#         messages=[
#             {"role": "system", "content": system},
#             {"role": "user", "content": user}
#         ],
#         temperature=0
#         )
#         text = completion.choices[0].message.content
#         qa_result_lst.append(text)
#         json.dump(text, file, ensure_ascii=False)
#         file.write('\n')
#         file.flush()
#         previous_question = text

100%|██████████| 232/232 [1:40:22<00:00, 25.96s/it]


In [ ]:
from langchain_core.documents import Document
from utils.util import get_project_root
import re
file_path = get_project_root() / "src" / "data" / "housing_pre_promise" / "housing_faq_chunks"


def load_txt_as_documents(path: str):
    docs = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # 빈 줄은 건너뜀
            docs.append(Document(page_content=line))
    return docs

docs = load_txt_as_documents(file_path)



In [ ]:
# from langchain_community.vectorstores import PGVector
# from langchain_openai import OpenAIEmbeddings
# import os 
# from tools.rag.db_collection_name import HOUSING_FAQ_KEY
# emb = OpenAIEmbeddings(model = "text-embedding-3-large")
# connection_url = os.getenv("POSTGRES_URL")
# store = PGVector.from_documents(
#         documents= docs,
#         embedding= emb, 
#         connection_string = connection_url,
#         collection_name= HOUSING_FAQ_KEY,
#         use_jsonb=True,
#         pre_delete_collection=False
# )

2025-11-02 22:54:39,994 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
